## Decision Tree

In [48]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt

### 1. Load dataset (5)

In [2]:
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
iris_df= iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2})
iris_df.head(2)

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


In [3]:
spambase_df = pd.read_csv("spambase.csv")
spambase_df.head(2)

,0,0.64,0.64.1,0.1,0.32,0.2,0.3,0.4,0.5,0.6,...,0.41,0.42,0.43,0.778,0.44,0.45,3.756,61,278,1
0,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
1,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1


In [4]:
# Dividing the df into X and y for both the datasets
X_iris =  iris_df.drop("class", axis=1).to_numpy()
y_iris = iris_df["class"].to_numpy()
X_spambase =  spambase_df.drop("1", axis=1).to_numpy()
y_spambase = spambase_df["1"].to_numpy()

In [5]:
# Splitting the data into train and test sets for both the datasets 
# X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.2, random_state=42)
# X_train_spambase, X_test_spambase, y_train_spambase, y_test_spambase = train_test_split(X_spambase, y_spambase, test_size=0.2, random_state=42)

### 2. - 6. Create_decision_tree method (40) - Finding best feature and threshold to split (20) - Finding the dominant class label in the node (5) - Calculating entropy (5) - Predict function (5)

In [49]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature 
        self.threshold = threshold 
        self.left = left 
        self.right = right 
        self.value = value 

    def is_leaf(self):
        return self.value is not None         

class decision_tree():  
    def __init__(self, n_min_samples=None):
        self.n_min_samples = n_min_samples
        self.root = None 
        
    def fit(self, X, y ):
        self.root =  self._build_tree(X, y) 
   
    # function that would facilitate in creating the child nodes based on the critriea 
    # its an iterative way of creating the tree so n_samp and n_feat could be the childern splits as well
    def _build_tree(self, X, y, depth=0):
        n_labels =  len(np.unique(y))
        
        # check the stopping critriea 
        # in this case the stopping critriea is based on the min of number of instances in a node
        if(n_labels == 1 or X.shape[0] < self.n_min_samples):
            leaf_value = self._common_label(y) # returning the class identifies
            return Node(value=leaf_value)

        idxs = np.arange(0, X.shape[1])
        np.random.shuffle(idxs)
        feat, thresh = self._best_split(X, y, idxs) # find the best split
   
        # create the child nodes 
        left_idx, right_idx = self._split(X[:, feat], thresh)
        left = self._build_tree(X[left_idx, :], y[left_idx], depth+1)
        right = self._build_tree(X[right_idx, :], y[right_idx], depth+1)
        
        return(Node(feat, thresh, left, right))
                               
    def _best_split(self, X, y, idxs):
        best_gain = -1
        best_feat, best_thresh = None, None # the feature that has the most information gain per given column

        for idx in idxs: # for all feature columns
            col = X[:, idx]
            thresholds = np.sort(np.unique(col)) 
            # Take the average of two consecutive numbers as the splitting threshold
            avg_thresholds = (thresholds[:-1] + thresholds[1:])/2  
        
            for thr in avg_thresholds: # information gain for all uniq vals
                gain  = self._information_gain(y, col, thr)

                if gain > best_gain: # finding the feature amongst the features that has the best gain 
                    best_gain = gain 
                    best_feat = idx
                    best_thresh = thr
                    
        return best_feat, best_thresh
        
    # Calculate the Information Gain for all the unique numbers in the column, 
    # then pick the one with the highest Information Gain.
    def _information_gain(self, y, x_column, threshold):
        #entropy of the parent 
        parent_ent = self._entropy(y)
        
        # creating the split
        left_idx, right_idx = self._split(x_column, threshold)
        if len(left_idx) == 0 or len(right_idx) == 0: 
            return 0
        
        # calculating the weighted avg. of the child entropy
        n = len(y)
        n_l, n_r = len(left_idx), len(right_idx)
        e_l, e_r = self._entropy(y[left_idx]), self._entropy(y[right_idx])
        child_ent = (n_l/n)*e_l + (n_r/n)*e_r
    
        #information gain formula 
        inf_gain = parent_ent - child_ent
        return inf_gain      
        
    def _split(self, x_column, split_thresh):
        left_idxs = np.where(x_column <= split_thresh)[0]
        right_idxs = np.where(x_column > split_thresh)[0]
        return left_idxs, right_idxs     
        
    def _entropy(self, y):
        _, occurence  = np.unique(y, return_counts=True) #array or uniq values and coressponding cts
        prob = occurence/len(y)
        # entropy  =  - sum_of_( prob * log(prob) )
        return -np.sum([p*np.log(p) for p in prob if p>0])      
    
    def _common_label(self, y):
        unique_labels, counts = np.unique(y, return_counts=True) #array or uniq values and coressponding cts
        common_index = np.argmax(counts) # both the arrays are connected 
        common_label = unique_labels[common_index]
        return common_label       
        
    def predict(self, X):
        predictions = [] # storing all the predictions
        for x in X:
            node = self.root # starting traversing from root
            while not node.is_leaf():
                if x[node.feature] <= node.threshold: # based on thr rule created while splitting 
                    node = node.left
                else:
                    node = node.right
            predictions.append(node.value)
        return np.array(predictions)


### 7.& 8. Testing on Iris Dataset (Accuracy for different Nmin) (10) - Testing on Spam email Dataset (Accuracy for different Nmin)	(10)

In [50]:
n_min_iris = np.array([5, 10, 15, 20])
n_samples_iris = np.ceil(n_min_iris/100 * X_iris.shape[0])
n_samples_iris

array([ 8., 15., 23., 30.])

In [51]:
n_min_spambase = np.array([5, 10, 15, 20, 25])
n_samples_spambase = np.ceil(n_min_spambase/100 * X_spambase.shape[0])
n_samples_spambase

array([ 230.,  460.,  690.,  920., 1150.])

In [52]:
# Common function for both
def pred_accuracy(n_samples, X, y):
    kf = KFold(n_splits=10, shuffle=True)
    
    accuracy_scores = []
    for n_min in n_samples:
        accuracies = []
        for train_index, test_index in kf.split(X):
            
            # Split the data into training and test sets
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
        
            model = decision_tree(n_min_samples=n_min) 
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
            accuracies.append(accuracy_score(y_test, y_pred, normalize=True))
        
        mean_accuracy = np.mean(accuracies)
        std_accuracy = np.std(accuracies)
        accuracy_scores.append({"Mean Accuracy": mean_accuracy, "Standard Deviation": std_accuracy})
        
    return accuracy_scores

In [53]:
len(X_iris), len(y_iris)

(150, 150)

In [54]:
scores_iris =  pred_accuracy(n_samples_iris, X_iris, y_iris)

print(f"For n_min values: {n_samples_iris}:")
print(f" The corresponding accuarcy scores and the standard deviations are:\n {scores_iris}")

For n_min values: [ 8. 15. 23. 30.]:
 The corresponding accuarcy scores and the standard deviations are:
 [{'Mean Accuracy': 0.96, 'Standard Deviation': 0.03265986323710903}, {'Mean Accuracy': 0.96, 'Standard Deviation': 0.04422166387140532}, {'Mean Accuracy': 0.9466666666666667, 'Standard Deviation': 0.04988876515698587}, {'Mean Accuracy': 0.9400000000000001, 'Standard Deviation': 0.06289320754704401}]


In [55]:
scores_spambase =  pred_accuracy(n_samples_spambase, X_spambase, y_spambase)

print(f"For n_min values: {n_samples_spambase}:")
print(f" The corresponding accuarcy scores and the standard deviations are:\n {n_samples_spambase}")

For n_min values: [ 230.  460.  690.  920. 1150.]:
 The corresponding accuarcy scores and the standard deviations are:
 [ 230.  460.  690.  920. 1150.]


In [56]:
scores_spambase

[{'Mean Accuracy': 0.9069565217391304,
  'Standard Deviation': 0.012442685235873194},
 {'Mean Accuracy': 0.8908695652173912,
  'Standard Deviation': 0.013534245579368163},
 {'Mean Accuracy': 0.866521739130435,
  'Standard Deviation': 0.017205529510790744},
 {'Mean Accuracy': 0.8432608695652174,
  'Standard Deviation': 0.02090689595986808},
 {'Mean Accuracy': 0.8321739130434782,
  'Standard Deviation': 0.019219161830695962}]